<a href="https://colab.research.google.com/github/HasankaRajakaruna/Kaggle-Projects/blob/main/Model_One.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
import pandas as pd
from datetime import datetime
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [166]:
train = pd.read_csv('/content/drive/MyDrive/Dataset/dataset_gbpusd_train.csv')
test = pd.read_csv('/content/drive/MyDrive/Dataset/dataset_gbpusd_test.csv')

In [167]:
train.shape

(21715, 10)

In [168]:
train.head()

,time,Unnamed: 0,open,high,low,close,tick_volume,spread,real_volume,target
0,2010-01-04 00:00:00,0,1.61210,1.61486,1.60852,1.61087,7471,24,0,2
1,2010-01-04 04:00:00,1,1.61087,1.61090,1.60691,1.60850,5966,24,0,2
2,2010-01-04 08:00:00,2,1.60851,1.62380,1.60554,1.61744,13700,24,0,2
3,2010-01-04 12:00:00,3,1.61748,1.62245,1.61375,1.61656,12921,24,0,2
4,2010-01-04 16:00:00,4,1.61651,1.61877,1.60822,1.60958,10789,23,0,2


In [169]:
test.shape

(384, 11)

In [170]:
test.head()

,Unnamed: 0.1,Unnamed: 0,time,open,high,low,close,tick_volume,spread,real_volume,target
0,0,0,2024-01-02 00:00:00,1.27205,1.27355,1.27173,1.27210,5319,0,0,2
1,1,1,2024-01-02 04:00:00,1.27210,1.27414,1.27100,1.27370,6880,0,0,2
2,2,2,2024-01-02 08:00:00,1.27370,1.27602,1.27174,1.27176,13263,0,0,2
3,3,3,2024-01-02 12:00:00,1.27176,1.27188,1.26217,1.26300,16548,0,0,2
4,4,4,2024-01-02 16:00:00,1.26291,1.26448,1.26108,1.26330,18454,0,0,2


In [124]:
dataset = pd.concat([train,test])

In [125]:
dataset.shape

(22099, 11)

In [126]:
duplicates = dataset[dataset.duplicated()]

print("Duplicate Rows:")
print(duplicates)

Duplicate Rows:
Empty DataFrame
Columns: [time, Unnamed: 0, open, high, low, close, tick_volume, spread, real_volume, target, Unnamed: 0.1]
Index: []


Calculate Target Value

In [127]:
array_with_zeros = np.zeros(len(dataset), dtype=int)

def calculate_target(data):

    for a in range(len(data)):
        max_value = data['high'].iloc[a]
        buy_price = data['open'].iloc[a]

        for b in range(a, a + 3, 1):
            if b < len(data):
                max_value = max(max_value, data['high'].iloc[b])
                diff_buy = (max_value - buy_price) * 10000

                if diff_buy >= 30:
                    array_with_zeros[a] = 1
                    break

calculate_target(dataset)


Target column

In [128]:
dataset.reset_index(drop=True, inplace=True)  # Reset the index of the DataFrame

target_column = pd.DataFrame({'target': array_with_zeros})

if 'target' in dataset.columns:
    dataset.drop(columns=['target'], inplace=True)

dataset = pd.concat([dataset, target_column], axis=1)

In [129]:
dataset.head()

,time,Unnamed: 0,open,high,low,close,tick_volume,spread,real_volume,Unnamed: 0.1,target
0,2010-01-04 00:00:00,0,1.61210,1.61486,1.60852,1.61087,7471,24,0,NaN,1
1,2010-01-04 04:00:00,1,1.61087,1.61090,1.60691,1.60850,5966,24,0,NaN,1
2,2010-01-04 08:00:00,2,1.60851,1.62380,1.60554,1.61744,13700,24,0,NaN,1
3,2010-01-04 12:00:00,3,1.61748,1.62245,1.61375,1.61656,12921,24,0,NaN,1
4,2010-01-04 16:00:00,4,1.61651,1.61877,1.60822,1.60958,10789,23,0,NaN,0


Heikinashi candlestics

In [130]:
def calculate_heikin_ashi(data):
    data["HA_Close"] = (data["open"] + data["high"] + data["low"] + data["close"]) / 4
    data["HA_Open"] = (data["open"].shift(1) + data["close"].shift(1)) / 2
    data["HA_High"] = data[["high", "close", "open"]].max(axis=1)
    data["HA_Low"] = data[["low", "close", "open"]].min(axis=1)

calculate_heikin_ashi(dataset)

In [131]:
dataset = dataset[['HA_Open', 'HA_High', 'HA_Low','HA_Close','target']]

In [132]:
dataset.head()

,HA_Open,HA_High,HA_Low,HA_Close,target
0,NaN,1.61486,1.60852,1.611588,1
1,1.611485,1.61090,1.60691,1.609295,1
2,1.609685,1.62380,1.60554,1.613822,1
3,1.612975,1.62245,1.61375,1.617560,1
4,1.617020,1.61877,1.60822,1.613270,0


In [133]:
close_prices = dataset['HA_Close'].values
high_prices = dataset['HA_High'].values
low_prices = dataset['HA_Low'].values
open_prices = dataset['HA_Open'].values

Indicators

In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

In [137]:
import talib as ta

In [138]:
#Ema indicator
ema_period5 = 5
dataset['EMA5'] = ta.EMA(close_prices, timeperiod=ema_period5)

ema_period10 = 10
dataset['EMA10'] = ta.EMA(close_prices, timeperiod=ema_period10)

ema_period20 = 20
dataset['EMA20'] = ta.EMA(close_prices, timeperiod=ema_period20)

ema_period50 = 50
dataset['EMA50'] = ta.EMA(close_prices, timeperiod=ema_period50)

ema_period100 = 100
dataset['EMA100'] = ta.EMA(close_prices, timeperiod=ema_period100)

ema_period200 = 200
dataset['EMA200'] = ta.EMA(close_prices, timeperiod=ema_period200)

Duplicate Data

In [139]:
duplicates = dataset[dataset.duplicated()]

print("Duplicate Rows:")
print(duplicates)

Duplicate Rows:
Empty DataFrame
Columns: [HA_Open, HA_High, HA_Low, HA_Close, target, EMA5, EMA10, EMA20, EMA50, EMA100, EMA200]
Index: []


In [ ]:
dataset.drop_duplicates(inplace=True)

Missing Values

In [140]:
missing_values = dataset.isnull().sum()

print("Columns with Missing Values:")
print(missing_values[missing_values > 0])

Columns with Missing Values:
HA_Open      1
EMA5         4
EMA10        9
EMA20       19
EMA50       49
EMA100      99
EMA200     199
dtype: int64


In [141]:
dataset.dropna(inplace=True)

In [142]:
dataset

,HA_Open,HA_High,HA_Low,HA_Close,target,EMA5,EMA10,EMA20,EMA50,EMA100,EMA200
199,1.566470,1.56809,1.56472,1.566120,1,1.569004,1.570448,1.570089,1.571661,1.580173,1.599697
200,1.565835,1.56695,1.55741,1.563025,0,1.567011,1.569098,1.569416,1.571323,1.579833,1.599332
201,1.563870,1.56732,1.55560,1.562138,1,1.565386,1.567833,1.568723,1.570962,1.579483,1.598962
202,1.562815,1.56580,1.55576,1.562115,0,1.564296,1.566793,1.568093,1.570615,1.579139,1.598595
203,1.563450,1.56327,1.54732,1.555213,0,1.561268,1.564687,1.566867,1.570011,1.578665,1.598164
...,...,...,...,...,...,...,...,...,...,...,...
22094,1.262180,1.26309,1.26151,1.262340,0,1.262512,1.262604,1.263069,1.265824,1.268108,1.268360
22095,1.262380,1.26257,1.26160,1.262233,0,1.262419,1.262536,1.262989,1.265683,1.267992,1.268299
22096,1.262380,1.26361,1.26092,1.262565,0,1.262468,1.262541,1.262949,1.265561,1.267884,1.268242
22097,1.262865,1.26445,1.26281,1.263325,0,1.262753,1.262684,1.262985,1.265473,1.267794,1.268193


Model Traning

In [171]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import joblib
from sklearn.metrics import accuracy_score

Train set and Test set

In [144]:
test_num_rows = test.shape[0]
test_data = dataset.tail(test_num_rows)
train_num_rows = dataset.shape[0]
train_data = dataset.head(train_num_rows-test_num_rows)

In [145]:
value_counts = train_data['target'].value_counts()
value_counts

target
0    11010
1    10506
Name: count, dtype: int64

In [146]:
#buy orders
bb_order = train_data[train_data['target'] == 1].tail(7000)
bb_order

,HA_Open,HA_High,HA_Low,HA_Close,target,EMA5,EMA10,EMA20,EMA50,EMA100,EMA200
7205,1.654590,1.65654,1.65506,1.655703,1,1.656438,1.657419,1.659524,1.665476,1.673467,1.682417
7206,1.655605,1.65885,1.65607,1.657478,1,1.656785,1.657430,1.659329,1.665162,1.673151,1.682169
7216,1.654395,1.65486,1.65366,1.654323,1,1.655809,1.656649,1.657813,1.662527,1.670267,1.679800
7217,1.654385,1.65593,1.65444,1.654920,1,1.655513,1.656335,1.657538,1.662228,1.669963,1.679553
7218,1.654655,1.65758,1.65425,1.655898,1,1.655641,1.656255,1.657382,1.661980,1.669685,1.679317
...,...,...,...,...,...,...,...,...,...,...,...
21707,1.276790,1.27752,1.27286,1.274620,1,1.277190,1.276512,1.274454,1.270708,1.266352,1.258473
21709,1.273490,1.27471,1.27207,1.273273,1,1.274964,1.275408,1.274221,1.270897,1.266619,1.258764
21710,1.273155,1.27637,1.27326,1.274898,1,1.274942,1.275315,1.274285,1.271054,1.266783,1.258925
21712,1.273430,1.27494,1.27042,1.272003,1,1.273652,1.274450,1.274004,1.271185,1.267018,1.259199


In [149]:
#sell orders
ss_order = train_data[train_data['target'] == 0].tail(7000)
ss_order

,HA_Open,HA_High,HA_Low,HA_Close,target,EMA5,EMA10,EMA20,EMA50,EMA100,EMA200
8160,1.486220,1.48768,1.47647,1.482522,0,1.486041,1.486911,1.487183,1.486721,1.489234,1.498079
8162,1.478780,1.48051,1.47120,1.475943,0,1.481345,1.483897,1.485498,1.486047,1.488793,1.497682
8163,1.476030,1.47337,1.46847,1.471580,0,1.478090,1.481658,1.484172,1.485480,1.488452,1.497422
8164,1.472240,1.47191,1.47035,1.471245,0,1.475808,1.479764,1.482941,1.484922,1.488111,1.497161
8165,1.471360,1.47241,1.47049,1.471450,0,1.474355,1.478253,1.481847,1.484393,1.487781,1.496906
...,...,...,...,...,...,...,...,...,...,...,...
21705,1.281105,1.28277,1.27910,1.280558,0,1.279219,1.276921,1.274168,1.270286,1.265967,1.258123
21706,1.280180,1.28054,1.27382,1.276985,0,1.278474,1.276933,1.274437,1.270549,1.266185,1.258311
21708,1.274050,1.27398,1.27124,1.273050,0,1.275810,1.275883,1.274320,1.270800,1.266485,1.258618
21711,1.274980,1.27728,1.27004,1.273545,0,1.274476,1.274993,1.274215,1.271152,1.266917,1.259070


In [150]:
new_dataset = pd.concat([bb_order, ss_order], ignore_index=True)
new_dataset

,HA_Open,HA_High,HA_Low,HA_Close,target,EMA5,EMA10,EMA20,EMA50,EMA100,EMA200
0,1.654590,1.65654,1.65506,1.655703,1,1.656438,1.657419,1.659524,1.665476,1.673467,1.682417
1,1.655605,1.65885,1.65607,1.657478,1,1.656785,1.657430,1.659329,1.665162,1.673151,1.682169
2,1.654395,1.65486,1.65366,1.654323,1,1.655809,1.656649,1.657813,1.662527,1.670267,1.679800
3,1.654385,1.65593,1.65444,1.654920,1,1.655513,1.656335,1.657538,1.662228,1.669963,1.679553
4,1.654655,1.65758,1.65425,1.655898,1,1.655641,1.656255,1.657382,1.661980,1.669685,1.679317
...,...,...,...,...,...,...,...,...,...,...,...
13995,1.281105,1.28277,1.27910,1.280558,0,1.279219,1.276921,1.274168,1.270286,1.265967,1.258123
13996,1.280180,1.28054,1.27382,1.276985,0,1.278474,1.276933,1.274437,1.270549,1.266185,1.258311
13997,1.274050,1.27398,1.27124,1.273050,0,1.275810,1.275883,1.274320,1.270800,1.266485,1.258618
13998,1.274980,1.27728,1.27004,1.273545,0,1.274476,1.274993,1.274215,1.271152,1.266917,1.259070


In [151]:
X_data=new_dataset.drop(['target'],axis=1)
y_data=new_dataset[['target']]

Scaling Data

In [152]:
scale = MinMaxScaler()
data_set_scaled = scale.fit_transform(X_data)
print(data_set_scaled)

[[0.9910972  0.98707039 0.99147354 ... 1.         1.         1.        ]
 [0.99275221 0.99089464 0.99309228 ... 0.99944557 0.99943093 0.99955037]
 [0.99077924 0.98428912 0.98922973 ... 0.99478712 0.99424893 0.99525932]
 ...
 [0.37060771 0.35373485 0.37631824 ... 0.30236764 0.26854621 0.2321172 ]
 [0.37212412 0.35919807 0.37439497 ... 0.30298897 0.26932303 0.23293585]
 [0.37080337 0.35750944 0.37861012 ... 0.30345521 0.27000858 0.23370634]]


**Model**

In [162]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score

x_train, x_test, y_train, y_test = train_test_split(data_set_scaled, y_data, test_size=0.2, random_state=42)

Decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model1 = DecisionTreeClassifier()
cross_val_score(model1, x_train, y_train, cv=10).mean()

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model2 = RandomForestClassifier(n_estimators=100)
cross_val_score(model2, x_train, y_train, cv=10).mean()

Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
model3 = GaussianNB()
cross_val_score(model3, x_train, y_train, cv=10).mean()

In [ ]:
pred3 = model3.predict(x_test)

In [ ]:
accuracy_score(y_test, pred3)

SVM

In [ ]:
from sklearn.svm import SVC
model4 = SVC()
cross_val_score(model4, x_train, y_train, cv=10).mean()

In [ ]:
pred4 = model4.predict(x_test)

In [ ]:
accuracy_score(y_test, pred4)

KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
correct_sum = []
for i in range(1,20):
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(x_train,y_train)
    pred = model.predict(x_test)
    correct = np.sum(pred == y_test)
    correct_sum.append(correct)

In [ ]:
result = pd.DataFrame(data=correct_sum)
result.index = result.index+1
result.T

In [ ]:
model5 = KNeighborsClassifier(n_neighbors=6)
cross_val_score(model5, x_train, y_train, cv=10).mean()

Testing the model

In [154]:
test_data

,HA_Open,HA_High,HA_Low,HA_Close,target,EMA5,EMA10,EMA20,EMA50,EMA100,EMA200
21715,1.274660,1.27355,1.27173,1.272358,1,1.273442,1.274005,1.273878,1.271452,1.267399,1.259623
21716,1.272075,1.27414,1.27100,1.272735,1,1.273207,1.273774,1.273769,1.271503,1.267505,1.259754
21717,1.272900,1.27602,1.27174,1.273305,0,1.273239,1.273689,1.273725,1.271573,1.267619,1.259889
21718,1.272730,1.27188,1.26217,1.267203,0,1.271227,1.272510,1.273104,1.271402,1.267611,1.259962
21719,1.267380,1.26448,1.26108,1.262943,0,1.268466,1.270770,1.272136,1.271070,1.267519,1.259991
...,...,...,...,...,...,...,...,...,...,...,...
22094,1.262180,1.26309,1.26151,1.262340,0,1.262512,1.262604,1.263069,1.265824,1.268108,1.268360
22095,1.262380,1.26257,1.26160,1.262233,0,1.262419,1.262536,1.262989,1.265683,1.267992,1.268299
22096,1.262380,1.26361,1.26092,1.262565,0,1.262468,1.262541,1.262949,1.265561,1.267884,1.268242
22097,1.262865,1.26445,1.26281,1.263325,0,1.262753,1.262684,1.262985,1.265473,1.267794,1.268193


In [155]:
duplicates = test_data[test_data.duplicated()]

print("Duplicate Rows:")
print(duplicates)

Duplicate Rows:
Empty DataFrame
Columns: [HA_Open, HA_High, HA_Low, HA_Close, target, EMA5, EMA10, EMA20, EMA50, EMA100, EMA200]
Index: []


In [156]:
missing_values = test_data.isnull().sum()

print("Columns with Missing Values:")
print(missing_values[missing_values > 0])

Columns with Missing Values:
Series([], dtype: int64)


In [157]:
value_coun = test_data['target'].value_counts()
value_coun

target
0    261
1    123
Name: count, dtype: int64

In [158]:
# Make predictions on the test dataset
X_test_data=test_data.drop(['target'],axis=1)
y_test_data=test_data[['target']]

In [160]:
scale = MinMaxScaler()
scaled_x_data = scale.fit_transform(X_test_data)
print(scaled_x_data)

[[0.65752571 0.54729923 0.60646736 ... 0.67349724 0.47043164 0.        ]
 [0.57697102 0.56416119 0.58419768 ... 0.67649866 0.48004565 0.01191551]
 [0.60267996 0.61789083 0.60677242 ... 0.68071631 0.49049624 0.02423047]
 ...
 [0.27485198 0.26321806 0.27669311 ... 0.32192669 0.51459517 0.78717167]
 [0.28996572 0.28722492 0.33435021 ... 0.31669369 0.50638087 0.78270297]
 [0.29479589 0.24206916 0.29194631 ... 0.30933739 0.49653657 0.77737451]]


Decision tree

In [ ]:
pred11 = model1.predict(X_test_data)
accuracy_score(y_test_data, pred11)

Random Forest

In [161]:
pred22 = model2.predict(X_test_data)
accuracy_score(y_test_data, pred22)


Accuracy: 0.3932291666666667


Naive Bayes

In [ ]:
pred33 = model3.predict(X_test_data)
accuracy_score(y_test_data, pred33)

SVM

In [ ]:
pred44 = model4.predict(X_test_data)
accuracy_score(y_test_data, pred44)

KNN

In [ ]:
pred55 = model5.predict(X_test_data)
accuracy_score(y_test_data, pred55)